# Introduction to Qibocal

In this tutorial, we will present `Qibocal`, an open-source software library to perform calibration and characterization of superconducting quantum devices within the Qibo framework. 
You will learn how to define a platform that emulate a physical one and how to execute some experiments.

First of all, let's install Qibocal and Qibolab with all the dependencies for the emulator.

In [ ]:
%%bash
pip install qibocal qibolab[emulator]

In [ ]:
from qibocal import create_calibration_platform
from IPython import display

from qibolab import ConfigKinds, DcChannel, IqChannel, Platform, Qubit, Parameters, PulseSequence, Delay, Sweeper, Delay, AveragingMode, Parameter
from qibolab.instruments.emulator import (
    DriveEmulatorConfig,
    EmulatorController,
    FluxEmulatorConfig,
    HamiltonianConfig,
)

### Platform definition

After importing the required modules, we define a configuration dictionary containing all setup parameters, and allocate the platform.

In [ ]:
config = {
  "settings": {
    "nshots": 1024,
    "relaxation_time": 0
  },
  "configs": {
    "emulator/bounds": {
    "kind": "bounds",
    "waveforms": 1000000,
    "readout": 50,
    "instructions": 200
  },
  "hamiltonian":{
    "transmon_levels": 2,
    "single_qubit": {
      "0": {
        "frequency": 5e9,
        "anharmonicity": -200e6,
        "t1": {
          "0-1": 1000 # <--- T1 value
        },
        "t2": {
          "0-1": 1900 # <--- T2 value
        }
      }
    },
    "kind": "hamiltonian"
  },
    "0/drive": {  
      "kind": "drive-emulator",
      "frequency": 4.995e9, # <--- Qubit drive frequency
      "rabi_frequency": 20e6,
      "scale_factor": 10
    },
    "0/drive12": {
      "kind": "drive-emulator",
      "frequency": 4.8e9,
      "rabi_frequency": 20e6,
      "scale_factor": 10
    },
  "0/probe": {
    "kind": "iq",
    "frequency": 5200000000.0
  },
  "0/acquisition": { 
    "kind": "acquisition",
    "delay": 0.0,
    "smearing": 0.0,
    "threshold": 0.0,
    "iq_angle": 0.0,
    "kernel": None
  }
  },
  "native_gates": {
    "single_qubit": {
      "0": {
        "RX": [
          [
            "0/drive",
            {
              "duration": 40,
              "amplitude": 0.12698,
              "envelope": {
                "kind": "gaussian",
                "rel_sigma": 0.2
              },
              "relative_phase": 0.0,
              "kind": "pulse"
            }
          ]
        ],
        "MZ": [
          [
            "0/acquisition",
            {
              "kind": "readout",
              "acquisition": {
                "kind": "acquisition",
                "duration": 100.0
              },
              "probe": {
                "duration": 100.0,
                "amplitude": 0.1,
                "envelope": {
                  "kind": "gaussian_square",
                  "rel_sigma": 0.2,
                  "width": 0.75
                },
                "relative_phase": 0.0,
                "kind": "pulse"
              }
            }
          ]
        ],
        "CP": None
      }
    }
    }
  }

Now, we use these configurations to create an emulator platform with one qubit.

In [ ]:
ConfigKinds.extend([HamiltonianConfig, DriveEmulatorConfig, FluxEmulatorConfig])

qubits = {}
channels = {}

for q in range(1):
    qubits[q] = qubit = Qubit.default(q)
    channels |= {
        qubit.drive: IqChannel(mixer=None, lo=None),
        qubit.flux: DcChannel(),
    }

# register the instruments
instruments = {
    "dummy": EmulatorController(address="0.0.0.0", channels=channels),
}

platform = Platform(
    name="emulator",
    parameters=Parameters(**config),
    instruments=instruments,
    qubits=qubits,
)

#### Further examples

Explore Qibolab & Qibocal itself for further emulator-based platforms, defined for internal use
- in [Qibolab's tests](https://github.com/qiboteam/qibolab/tree/main/tests/instruments/emulator/platforms), used to test the emulator itself
- in [Qibocal's platforms library](https://github.com/qiboteam/qibocal/tree/main/platforms), used in the protocols development

## Rabi Experiment

The goal of the Rabi experiment is to tune the amplitude (duration) of the drive pulse, in order
to excite the qubit from the ground state up to state $\ket{1}$.

In the Rabi experiment, the qubit is probed with a drive pulse at the qubit frequency $w_{01}$
before measuring. This pulse sequence is repeated multiple times changing the amplitude (duration) of the pulse.
The qubit starts in the ground state, sweeping one of the two parameters of the drive pulse, the probability of being in the first
excited state increases following a sinusoidal pattern.

For the amplitude version, we expect:

$$ p_e(t) =\frac{1}{2} \left( 1 - \cos\left(\Omega_R t \right) \right) $$

### Acquisition

In [ ]:
from qibocal.protocols import rabi_amplitude

# define qubits where the protocols will be executed
targets = [0]

# define protocol parameters
params = rabi_amplitude.parameters_type.load(dict(
    min_amp=0.01,
    max_amp=0.2,
    step_amp=0.02,
    nshots=2000,
    pulse_length=40,
))

# acquire
platform.connect()

data, acquisition_time = rabi_amplitude.acquisition(
    params=params,
    platform=platform,
    targets=targets
)

platform.disconnect()

### Fit and Report

In [ ]:
# post-processing
results, fit_time = rabi_amplitude.fit(data=data)

# visualize the results
plots, table = rabi_amplitude.report(data=data, fit=results, target=targets[0])

# update the platform with the new results
rabi_amplitude.update(
    results = results,
    platform = platform,
    target = targets[0],
)

plots[0].show()

In [ ]:
# Display the results table
display.HTML(table)

## Ramsey experiment

To measure the decoherence time and the detuning frequency of a qubit, it is possible to 
perform a Ramsey experiment.
In this protocol two short $\pi /2$ are sent separated by a waiting time $\tau$.
The first $\pi /2$ pulse will project the qubit from $\ket{0}$ onto the $XY$
plane of the Bloch sphere. During the waiting time $\tau$ the qubit will decohere around
the z axis, after the second $\pi /2$ we peform a measurement.
By sweeping the waiting time $\tau$, we should observe an exponential decay
in the probability of measuring $\ket{1}$ which will give us an estimate of $T_2*$.
In the case where there is a difference between the qubit frequency and the frequency of
the drive pulse, the final state will show a sinusoidal dependence from which we can extract
a correction on the qubit frequency.

$$ p_e = \frac{1}{2} + \frac{1}{2} e^{-\tau/T_2} \cos(\Delta \omega \pm \pi/2)$$

In order to obtain the correction on the qubit frequency reliably usually the
drive pulse it is detuned on purpose to generate oscillations.

### Acquisition

In [ ]:
from qibocal.protocols import rabi_amplitude, ramsey

params_ramsey = ramsey.parameters_type.load(dict(
    delay_between_pulses_start = 0,
    delay_between_pulses_end = 300,
    delay_between_pulses_step = 20,
    detuning = 10_000_000, 
    nshots = 200,
))

platform.connect()
data_ramsey, _ = ramsey.acquisition(
    params=params_ramsey,
    platform=platform,
    targets=targets
)
platform.disconnect()

### Fit and Report

## T1 experiment

Due to coupling to the environment the qubit in state $\ket{1}$
will decay to the state $\ket{0}$. To measure such decay we can perform
a simple experiment by initializing the qubit in $\ket{1}$ through a
$\pi$-pulse previously calibrated, we then measure the population of
the excited state for different waiting times $\Delta \tau$.
We expect to measure an exponential decay which is fitted with the following formula:

$$ p_e(t) = A + B  e^{ - t / T_1} $$

$A$ and $B$ are introduced to improve the performance of the fit.

### Acquisition

### Fit and report

## T2 experiment

The acquisition for the $T_2$ experiment is the same one as for the ``Ramsey`` experiment,
the only difference is that in this experiment the drive pulse is not detuned. This protocol
is assuming that any error on the drive frequency has been already corrected through a ``Ramsey`` experiment.
For this reason we expect to see just an exponential behavior:

$$p_e(t) = A + B  e^{ - t / T_2}$$

The reason why there are two distinct experiment is that in order to correct the drive frequency
if the pulse is detuned enough we can proceed with short scans, while to extract reliably  $T_2$
it will be necessary to perform longer scans.

### Acquisition

### Fit and report

> **Hint:** the $T_2$ fit is assuming an exponential decay, as described above. If residual oscillations are observed, better to find that frequency, and then "remove" it 😉

## References

- Qibocal website: https://qibo.science/qibocal/stable/index.html
- For more info about Qibocal protocols: https://qibo.science/qibocal/stable/protocols/index.html
- A Quantum Engineer's Guide to Superconducting Qubits: https://arxiv.org/abs/1904.06560